#### Collecting and storing data from NYTimes API

In [9]:
import requests, configparser, os, sys, json, glob
from nltk.tag import pos_tag
import operator

In [14]:
# Collecting data

config = configparser.ConfigParser()
config.readfp(open(r'../extra/config.txt'))

C:\Users\Rainbow\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: DeprecationWarning: This method will be removed in future versions.  Use 'parser.read_file()' instead.


In [15]:
i = 1
try:
    topic_index_dict = {}
    while i < 2:
        try:
            print('page '+str(i))
            uri_path_list = []
            path = ''

            #creating the url to hit
            for link in config['uri']:
                uri_path_list.append(config.get('uri',link))
                break;            

            for directory in config['directory_path']:
                if path == '':
                    path = config.get('directory_path',directory)
                else:
                    path = path + '\\' + config.get('directory_path',directory)



            for key in config['api_key']:
                uri_path_list.append('?api_key='+config.get('api_key',key))
                break;   

            for param in config['parameters']:
                uri_path_list.append('&'+param+'='+config.get('parameters',param))

            uri_path_list.append('&page=' + str(i) )
            uri_path = ''.join(uri_path_list);

            #hitting the uri
            resp = requests.get(uri_path)
            request_data = (resp).json();  
            if 'errors' in request_data.keys():
                raise Exception(request_data['errors'])
            elif 'message' in request_data.keys():
                raise Exception(request_data['message'])


            try:    
                if( 'response' in request_data.keys() ):
                    response = request_data['response']
                    if( 'docs' in response.keys() ):
                        docs = response['docs']
                        count = 0
                        for doc in docs:
                            count += 1
                            print('doc '+str(count)+' in page')
                            #print(doc.keys())
                            #dict_keys(['web_url', 'snippet', 'lead_paragraph', 'abstract', 'print_page', 'blog', 
                            #'source', 'multimedia', 'headline', 'keywords', 'pub_date', 'document_type', 'news_desk', 
                            #'section_name', 'subsection_name', 'byline', 'type_of_material', '_id', 'word_count', 
                            #'slideshow_credits'])
                            if( 'keywords' in doc.keys() ):
                                keywords = doc['keywords']
                                yes_cali = False 
                                try:
                                    for key_set in keywords:
                                        if 'name' in key_set.keys() and 'value' in key_set.keys():
                                            name = key_set['name']
                                            value = key_set['value']
                                            if name.lower() == 'glocations' and value.lower() == 'california':
                                                yes_cali = True
                                except Exception as error:
                                    print(repr(error))            
                                        
                                if yes_cali:
                                    for key_set in keywords:
                                        #print('Printing keys before error')
                                        #print(key_set.keys())
                                        try:
                                            if 'name' in key_set.keys() and 'value' in key_set.keys():
                                                name = key_set['name']
                                                value = key_set['value']
                                                if name.lower() == 'subject':
                                                    if not os.path.exists(path+'\\'+value):
                                                        os.makedirs(path+'\\'+value)
                                                    file_path = path+'\\'+value+'\\'+doc.get('_id')+".json"

                                                    #checking if file exists
                                                    if not os.path.exists(file_path):
                                                        #updating the counter    
                                                        if value in topic_index_dict:
                                                            topic_index_dict[value] = topic_index_dict[value] + 1
                                                        else:
                                                            topic_index_dict[value] = 1
                                                        #dumping the file
                                                        #print(file_path+' writing')
                                                        with open(file_path, "w") as outfile:
                                                            json.dump(doc, outfile, indent=4)                                                        
                                                    #else: 
                                                        #print(file_path+' exists')
                                        except Exception as error:
                                            print('Caught --> : ' + repr(error))
            except ValueError:
                print("error :", sys.exc_info()[0])
                break
            #print(path)        

            i = i+1
        except Exception as error:
            print('Caught this error in while: ' + repr(error))
            continue  
except Exception as error:
    print('Caught this error: ' + repr(error))
print('Completed collecting and storing!')

page 1
doc 1 in page
doc 2 in page
doc 3 in page
doc 4 in page
doc 5 in page
doc 6 in page
doc 7 in page
doc 8 in page
doc 9 in page
doc 10 in page
Completed collecting and storing!
